In [20]:
import pandas as pd
import numpy as np
import os
import random

import torch
import torch.nn as nn
import os
import yaml
import itertools
import json
import traceback

from datetime import datetime
from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from classification_rnn import ClassificationRNN, DEVICE
from seed import set_seed
from config import flatten_config


In [6]:
BASE = "../../data/aisdk/processed"


# 1 - Create the windows

In [15]:
# TODO: ADD A CLASSIFIER HERE!!
def _get_cluster_id_for_segment(traj_id):
    """
    Assign cluster to a given segment
    PLACEHOLDER FOR NOW
    """
    return random.randint(0, 9)

In [16]:
def make_past_future_windows_np(
    past_len=30,
    future_len=30,
    step=1,
    input_path="data/aisdk/processed/train_trajectories.npz",
    output_path="data/aisdk/processed/windows/train_trajectories.npz"
):
    """
    Load standardized, sorted trajectory data from an npz file,
    create past/future sliding windows, and save everything as NumPy arrays
    in a single .npz file:

        - past:    (N, past_len, num_features)
        - future:  (N, future_len, num_features)
        - traj_id: (N,) - original trajectory ID for each window

    """
    
    print(f"Loading trajectories from {input_path} ...")
    data = np.load(input_path, allow_pickle=True)
    trajs = data["trajectories"]  # (N,) array of variable-length arrays (T_i, F)
    traj_ids = data["traj_ids"]   # (N,)
    feature_cols = data["feature_cols"]  # feature names
    num_traj = len(trajs)
    print(f"  → Found {num_traj} trajectories")
    print(f"  → Features: {feature_cols}")

    total_len = past_len + future_len

    past_list = []
    future_list = []
    traj_list = []
    cluster_list = []

    for i, traj in enumerate(trajs):
        # traj: (T_i, F)
        T = traj.shape[0]
        if T < total_len:
            continue  # too short for one window

        # number of windows for this trajectory (with stride `step`)
        num_windows = (T - total_len) // step + 1

        # assign a cluster
        cid = _get_cluster_id_for_segment(traj_ids[i])

        for w in range(num_windows):
            start = w * step
            mid   = start + past_len
            end   = mid + future_len

            past_window = traj[start:mid]   # (past_len, F)
            future_window = traj[mid:end]   # (future_len, F)

            past_list.append(past_window)
            future_list.append(future_window)
            traj_list.append(traj_ids[i])
            cluster_list.append(cid)

        if (i + 1) % 500 == 0:
            print(f"  → Processed {i+1}/{num_traj} trajectories, {len(past_list)} windows so far")

    if not past_list:
        raise RuntimeError("No windows generated. "
                           "Check past_len, future_len, and trajectory lengths.")

    print("Stacking windows into numpy arrays...")
    past = np.stack(past_list)     # (N, past_len, F)
    future = np.stack(future_list) # (N, future_len, F)
    traj_id = np.array(traj_list)  # (N,)
    cluster = np.array(cluster_list)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    print(f"Saving windows to {output_path} ...")
    np.savez_compressed(
        output_path,
        past=past,
        future=future,
        cluster=cluster,
        traj_id=traj_id,
        feature_cols=feature_cols,
        past_len=past_len,
        future_len=future_len,
        step=step,
    )

    print("\nDONE!")
    print(f"  → Total windows: {past.shape[0]:,}")
    print(f"  → Shape - past: {past.shape}, future: {future.shape}")
    print(f"  → Output saved to: {output_path}")
    
    # Summary statistics
    unique_trajs = np.unique(traj_id)
    print(f"  → Windows span {len(unique_trajs)} unique trajectories")
    print(f"  → Avg windows per trajectory: {len(past_list) / len(unique_trajs):.1f}")

In [17]:
make_past_future_windows_np(
    input_path=os.path.join(BASE, "train_trajectories.npz"),
    output_path=os.path.join(BASE, "windows/train_trajectories.npz"),
)

make_past_future_windows_np(
    input_path  = os.path.join(BASE, "val_trajectories.npz"),
    output_path = os.path.join(BASE, "windows/val_trajectories.npz"),
)

make_past_future_windows_np(
    input_path  = os.path.join(BASE, "test_trajectories.npz"),
    output_path = os.path.join(BASE, "windows/test_trajectories.npz")
)

Loading trajectories from ../../data/aisdk/processed/train_trajectories.npz ...
  → Found 359 trajectories
  → Features: ['UTM_x' 'UTM_y' 'SOG' 'v_east' 'v_north']
Stacking windows into numpy arrays...
Saving windows to ../../data/aisdk/processed/windows/train_trajectories.npz ...

DONE!
  → Total windows: 171,461
  → Shape - past: (171461, 30, 5), future: (171461, 30, 5)
  → Output saved to: ../../data/aisdk/processed/windows/train_trajectories.npz
  → Windows span 315 unique trajectories
  → Avg windows per trajectory: 544.3
Loading trajectories from ../../data/aisdk/processed/val_trajectories.npz ...
  → Found 77 trajectories
  → Features: ['UTM_x' 'UTM_y' 'SOG' 'v_east' 'v_north']
Stacking windows into numpy arrays...
Saving windows to ../../data/aisdk/processed/windows/val_trajectories.npz ...

DONE!
  → Total windows: 36,719
  → Shape - past: (36719, 30, 5), future: (36719, 30, 5)
  → Output saved to: ../../data/aisdk/processed/windows/val_trajectories.npz
  → Windows span 66 uni

# 1 - Fetch the data

In [ ]:
train_traj = np.load(os.path.join(BASE, "windows/train_trajectories.npz"))
val_traj = np.load(os.path.join(BASE, "windows/val_trajectories.npz"))
test_traj = np.load(os.path.join(BASE, "windows/test_trajectories.npz"))

X_train, X_val = train_traj["past"], val_traj["past"]
y_train, y_val = train_traj["cluster"], val_traj["cluster"]

# %% Convert to PyTorch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_val_t   = torch.tensor(X_val,   dtype=torch.float32)

y_train_t = torch.tensor(y_train, dtype=torch.long)
y_val_t   = torch.tensor(y_val,   dtype=torch.long)

# Create data loaders
print(f"\nTensor shapes:")
print(f"  X_train_t: {X_train_t.shape}")
print(f"  X_val_t:   {X_val_t.shape}")
print(f"  y_train_t: {y_train_t.shape}")
print(f"  y_val_t:   {y_val_t.shape}")

# %% Create data loaders
def make_loaders(batch_size):
    train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

# 2 - Train
## 2.1 - Define functions for one run

In [14]:
def _train_one_run(cfg, train_loader, val_loader):
    
    device = cfg["device"]

    model = ClassificationRNN(
        input_size=cfg["input_size"],
        hidden_size=cfg["hidden_size"],
        num_classes=cfg["num_classes"], 
    ).to(device)

    opt = torch.optim.Adam(
        model.parameters(), 
        lr=cfg["lr"], 
        weight_decay=cfg["weight_decay"]
    )

    crit = nn.CrossEntropyLoss()

    best_val_loss = float("inf")
    best_val_acc  = 0.0

    # %% Training loop
    for epoch in range(cfg["epochs"]):
    
        # ------ Training phase ------
        model.train()
        train_loss_total = 0.0
        train_correct = 0
        train_samples = 0
        
        for xb, yb in tqdm(train_loader, desc=f"Epoch {epoch}/{cfg["epochs"]} [Train]"):
            xb = xb.to(device)  # (B, seq_len, num_features)
            yb = yb.to(device)  # (B,)

            opt.zero_grad()
            logits = model(xb)  # (B, num_classes)
            loss = crit(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), cfg["grad_clip"])
            opt.step()

            train_loss_total += loss.item() #* xb.size(0)

            # Calculate accuracy
            preds = logits.argmax(dim=1)  # (B,)
            train_correct += (preds == yb).sum().item()
            train_samples += yb.size(0)

        train_loss = train_loss_total / len(train_loader)
        train_acc = train_correct / train_samples if train_samples > 0 else 0.0

        # ------ Validation ------
        model.eval()
        val_loss_total = 0.0
        val_correct = 0
        val_samples = 0

        with torch.no_grad():
            for xb, yb in tqdm(val_loader, desc=f"Epoch {epoch}/{cfg["epochs"]} [Val]  "):
                xb = xb.to(device)
                yb = yb.to(device)

                logits = model(xb)
                loss = crit(logits, yb)
                val_loss_total += loss.item() #*xb.size(0)

                preds = logits.argmax(dim=1)
                val_correct += (preds == yb).sum().item()
                val_samples += yb.size(0)

        val_loss = val_loss_total / len(val_loader)
        val_acc = val_correct / val_samples if val_samples > 0 else 0.0
        
        # Update best validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print(f"  → New best validation loss: {best_val_loss:.4f}")
        
        return best_val_loss, best_val_acc


## 2.2 - Hyperparameter tuning

In [ ]:
# Define key metrics
input_size = X_train.shape[-1]
n_classes = int(y_train.max() + 1)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def hyperparameter_search_classification(
    device=device,
    input_size=input_size,
    num_classes=n_classes,
    search_type='grid',
    save_dir='../../checkpoints/hyperparameter_results_classification'
):
    os.makedirs(save_dir, exist_ok=True)
    
    print("="*70)
    print("HYPERPARAMETER TUNING FOR CLASSIFICATION RNN")
    print("="*70)
    
    # -------- 1) Define search space --------
    if search_type == 'grid':
        param_grid = {
            # Tune these:
            'hidden_size'   : [64, 128],
            'num_layers'    : [1, 2],
            'dropout'       : [0.0, 0.3],
            'lr'            : [1e-4, 3e-4, 1e-3],
            'weight_decay'  : [0.0, 1e-4],
            
            # Fixed:
            'batch_size'    : [256],
            'epochs'        : [20],
            #'patience'     : [5],
            #'min_delta'    : [1e-4],
            'grad_clip'     : [1.0],
        }

        # total combos: 2 * 2 * 2 * 3 = 24
        
    elif search_type == 'quick':
        # Smaller search – good for smoke-testing
        param_grid = {
            'hidden_size'   : [64, 128],
            'num_layers'    : [1],
            'dropout'       : [0.0, 0.3],
            'lr'            : [3e-4],
            'weight_decay'  : [0.0, 1e-4],

            
            'batch_size'    : [256],
            'epochs'        : [15],
            #'patience'     : [3],
            #'min_delta'    : [1e-4],
            'grad_clip'     : [1.0],
        }
        # total combos: 2 * 1 * 2 * 1 = 4
        
    else:
        raise ValueError(f"Unknown search_type: {search_type}")
    
    # Generate combinations
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    combinations = list(itertools.product(*values))
    param_combinations = [dict(zip(keys, combo)) for combo in combinations]
    
    print(f"{search_type.capitalize()} Search: {len(param_combinations)} combinations")
    # super rough estimate: 5 min / combo like your friend
    print(f"Estimated time: {len(param_combinations) * 5} minutes")
    print("="*70)
    
    # -------- 2) Tracking --------
    results = []
    best_score = float('inf')   # here: best_val_loss
    best_params = None
    start_time = datetime.now()
    
    # -------- 3) Main search loop --------
    for idx, params in enumerate(param_combinations):
        print(f"\n{'='*70}")
        print(f"Trial {idx+1}/{len(param_combinations)}")
        elapsed_min = (datetime.now() - start_time).total_seconds() / 60
        print(f"Time elapsed: {elapsed_min:.1f} min")
        print(f"{'='*70}")
        
        print("Testing config:")
        print(f"  hidden_size   = {params['hidden_size']}")
        print(f"  num_layers    = {params['num_layers']}")
        print(f"  dropout       = {params['dropout']}")
        print(f"  lr            = {params['lr']}")
        print(f"  weight_decay  = {params['weight_decay']}")
        print(f"  batch_size    = {params['batch_size']}")
        print(f"  epochs        = {params['epochs']}")
        print()
        
        try:
            # ---- Build config dict for your existing _train_one_run ----
            cfg = {
                "device": device,
                "input_size": input_size,
                "num_classes": num_classes,
                
                # training hyperparams
                "hidden_size": params["hidden_size"],
                "num_layers": params["num_layers"],
                "dropout"   : params["dropout"],
                "lr"        : params["lr"],
                "weight_decay": params["weight_decay"],
                "batch_size": params["batch_size"],
                "epochs"    : params["epochs"],
                #"patience": params["patience"],
                #"min_delta": params["min_delta"],
                "grad_clip": params["grad_clip"],
            }
            
            train_loader, val_loader = make_loaders(cfg["batch_size"])

            model, train_losses, val_losses, val_accs = _train_one_run(cfg, train_loader, val_loader)
            
            # ---- Extract metrics ----
            val_losses = list(val_losses)
            best_val_loss = float(min(val_losses))
            best_epoch = int(val_losses.index(best_val_loss) + 1)
            
            if val_accs is not None:
                val_accs = list(val_accs)
                best_val_acc = float(val_accs[best_epoch - 1])
            else:
                best_val_acc = float("nan")
            
            final_train_loss = float(train_losses[best_epoch - 1])
            overfit_ratio = best_val_loss / max(final_train_loss, 1e-6)
            
            score = best_val_loss  # primary metric: lower is better
            
            result = {
                **params,
                'best_val_loss': best_val_loss,
                'best_val_acc': best_val_acc,
                'best_epoch': best_epoch,
                'final_train_loss': final_train_loss,
                'overfit_ratio': overfit_ratio,
                'score': score,
                'trial': idx + 1,
            }
            results.append(result)
            
            print(f"\nResults:")
            print(f"  Best Val Loss: {best_val_loss:.4f} (epoch {best_epoch})")
            if not np.isnan(best_val_acc):
                print(f"  Best Val Acc : {best_val_acc:.4f}")
            print(f"  Train Loss @best: {final_train_loss:.4f}")
            print(f"  Overfit ratio: {overfit_ratio:.2f}")
            print(f"  Score (val_loss): {score:.4f}")
            
            if score < best_score:
                best_score = score
                best_params = params.copy()
                print(f"  ⭐ NEW BEST CONFIG FOUND!")
                
                
                # Save best model
                torch.save(model.state_dict(), os.path.join(save_dir, "best_model.pth"))
            
            # Clean up
            del model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        except Exception as e:
            print(f"  ❌ FAILED: {str(e)}")
            traceback.print_exc()
            result = {**params, 'error': str(e), 'trial': idx + 1}
            results.append(result)
    
    # -------- 4) Save & summarize --------
    if len(results) == 0:
        print("\n❌ No trials completed!")
        return None, None
    
    df = pd.DataFrame(results)
    
    # Sort by score (val loss)
    if 'score' in df.columns:
        df = df.sort_values('score', na_position='last')
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_path = os.path.join(save_dir, f"results_{timestamp}.csv")
    df.to_csv(csv_path, index=False)
    
    if best_params is None:
        print("\n❌ All trials failed! Check error messages above.")
        return df, None
    
    json_path = os.path.join(save_dir, f"best_params_{timestamp}.json")
    with open(json_path, 'w') as f:
        json.dump(best_params, f, indent=2)
    
    total_time = (datetime.now() - start_time).total_seconds() / 60
    print(f"\n{'='*70}")
    print("HYPERPARAMETER SEARCH COMPLETE")
    print("="*70)
    print(f"Total time: {total_time:.1f} minutes")
    successful = [r for r in results if 'error' not in r]
    print(f"Successful trials: {len(successful)}/{len(results)}")
    
    print(f"\nBest parameters (score={best_score:.4f}):")
    for k, v in best_params.items():
        print(f"  {k}: {v}")
    
    print(f"\nResults saved to: {csv_path}")
    print(f"Best params saved to: {json_path}")
    print(f"Best model saved to: {os.path.join(save_dir, 'best_model.pth')}")
    
    # Show top 5 successful trials
    successful_df = df[df['score'].notna()]
    top_cols = [
        'hidden_size', 'num_layers', 'dropout', 'learning_rate',
        'batch_size', 'best_val_loss', 'best_val_acc', 'score'
    ]
    print("\nTop 5 configurations:")
    print(successful_df[top_cols].head())
    
    return df, best_params


In [ ]:
# Run the training
df_cls, best_params_cls = hyperparameter_search_classification(
    device=device,
    input_size=input_size,
    num_classes=n_classes,
    search_type='grid',
    save_dir='../../checkpoints/hyperparameter_results_classification' 
)

HYPERPARAMETER TUNING FOR CLASSIFICATION RNN
Grid Search: 48 combinations
Estimated time: 240 minutes

Trial 1/48
Time elapsed: 0.0 min
Testing config:
  hidden_size   = 64
  num_layers    = 1
  dropout       = 0.0
  lr            = 0.0001
  weight_decay  = 0.0
  batch_size    = 256
  epochs        = 20



Epoch 0/20 [Train]:  44%|████▍     | 295/669 [01:03<01:20,  4.66it/s]


KeyboardInterrupt: 

In [ ]:
# keys = list(param_grid.keys())
# all_combos = list(product(*[param_grid[k] for k in keys]))


# results = []  # to store (config, best_val_loss, best_val_acc)

# print(f"Total combinations: {len(all_combos)}")


# for i, values in enumerate(all_combos):
#     # ---- build config for this combo ----
#     cfg = base_config.copy()
#     for k, v in zip(keys, values):
#         cfg[k] = v

#     # make loaders for this batch size
#     train_loader, val_loader = make_loaders(cfg["batch_size"])

#     # ---- W&B run (optional but recommended) ----
#     wandb.init(
#         project="dl-maritime-classification-grid",
#         config=cfg,
#         name=f"grid_run_{i}",
#         reinit=True,
#     )

#     best_val_loss, best_val_acc = _train_one_run(wandb.config, train_loader, val_loader)

#     wandb.log({
#         "best_val_loss": best_val_loss,
#         "best_val_acc": best_val_acc,
#     })
#     wandb.finish()

#     results.append((cfg.copy(), float(best_val_loss), float(best_val_acc)))

Total combinations: 96


Epoch 0/20 [Val]  : 100%|██████████| 574/574 [00:13<00:00, 42.04it/s]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


  → New best validation loss: 2.3203


best_val_acc,▁
best_val_loss,▁
best_val_acc,0
best_val_loss,2.32034


Epoch 0/20 [Train]:  70%|██████▉   | 1873/2679 [02:13<00:57, 13.99it/s]


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x168ebdc70>> (for post_run_cell), with arguments args (<ExecutionResult object at 32fb2a7b0, execution_count=10 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 32fb2a960, raw_cell="keys = list(param_grid.keys())
all_combos = list(p.." transformed_cell="keys = list(param_grid.keys())
all_combos = list(p.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/Users/helgamariamagnusdottir/Documents/dtu/deep_learning/DL-group-63-P29/src/models/train_classification_rnn.ipynb#Y104sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: Connection lost

## 3.3 - Save model and plot

In [ ]:
# %% Save model
os.makedirs(MODEL_DIR, exist_ok=True)
model_path = os.path.join(MODEL_DIR, "classification_rnn_model.pt")
torch.save(model.state_dict(), model_path)
print(f"\nModel saved to: {model_path}")

# Optional: save to WandB
# if wandb.run is not None:
#     wandb.save(model_path)
#     wandb.run.summary["best_val_loss"] = best_val_loss

# %% Plot training history (optional)
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
ax1.plot(train_losses, label='Train Loss', marker='o')
ax1.plot(val_losses, label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True)

# Accuracy plot
ax2.plot(train_accs, label='Train Accuracy', marker='o')
ax2.plot(val_accs, label='Val Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# %% Training summary
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Total epochs: {cfg['epochs']}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Final train loss: {train_losses[-1]:.4f}")
print(f"Final train accuracy: {train_accs[-1]:.4f}")
print(f"Final val loss: {val_losses[-1]:.4f}")
print(f"Final val accuracy: {val_accs[-1]:.4f}")
print(f"\nModel saved to: {model_path}")
print("="*60)